![](profitable-app-profiles.jpg)

# Perfis de aplicativos lucrativos na App Store e Google Play

Para aplicativos gratuitos, a principal fonte de renda vem de anúncios dentro do app. Dessa forma, o objetivo desse projeto é analisar os perfis de aplicativos lucrativos e assim entender que tipo de app provavelmente atrairá mais usuários - quanto mais usuários visualizarem e interagirem com os anúncios, melhor.

---

## Explorando os dados

Vamos utilizar dois datasets como amostra:
* [App Store](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv)
* [Google Play](https://dq-content.s3.amazonaws.com/350/AppleStore.csv)

O primeiro dataset contém dados sobre aproximadamente 10.000 aplicativos Android do Google Play, os dados foram coletados em agosto de 2018. O segundo possui dados de mais ou menos 7.000 aplicativos iOS da App Store (os dados foram coletados em julho de 2017).

### Transformando os dados em uma lista
Vamos abrir os dois datasets e depois salvar cada um em uma lista separada.

In [1]:
from csv import reader
opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
data_apple = list(read_file)[1:]

In [2]:
from csv import reader
opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
data_google = list(read_file)[1:]

### Extraindo características dos dados
Aqui a ideia é criar uma função que possa ser usada diversas vezes para mostrar as linhas do dataset de uma maneira mais legível. Além disso, a função irá mostrar a quantidade de linhas e colunas do dataset escolhido para assim entender de maneira rápida qual o tamanho.

In [3]:
def explore_data(dataset, start, end, rows_and_columns=False): # start e end são os índices do dataset
    dataset_slice = dataset[start:end]  
    for row in dataset_slice: # para cada linha da lista dataset_slice
        print(row) # imprime a linha
        print('\n') # adiciona uma quebra de linha

    if rows_and_columns: # se rows_and_columns for True
        print('Quantidade de linhas:', len(dataset)) # imprime a quantidade de linhas
        print('Quantidade de colunas:', len(dataset[0])) # imprime a quantidade de colunas

In [4]:
explore_data(data_apple, 0, 5, True) # mostra as primeiras 5 linhas do dataset

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


Quantidade de linhas: 7197
Quantidade de colunas: 16


O dataset da App Store possui 7197 apps cadastrados. Colunas interessantes para ficar de olho: 'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', e 'prime_genre'.

In [5]:
explore_data(data_google, 0, 5, True) # mostra as primeiras 5 linhas do dataset

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


Quantidade de linhas: 10841
Quantidade de colunas: 13


Já o dataset do Google Play, contém 10841 aplicativos. As colunas que podem servir ao propósito desta análise são 'App', 'Category', 'Reviews', 'Installs', 'Type', 'Price', e 'Genres'.

Para saber mais sobre os datasets acesse as infos no Kaggle:
* [App Store](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps)
* [Google Play](https://www.kaggle.com/datasets/lava18/google-play-store-apps)

---

## Limpando os dados

Para essa análise, vamos trabalhar somente com aplicativos que são gratuitos e estão em inglês.

O que é necessário fazer nessa etapa:
* Remover apps duplicados
* Remover as linhas que contém dados de aplicativos que não são em inglês
* Remover as linhas que contém dados de aplicativos que não são gratuitos

Ao ler as [discussões](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) sobre o dataset do Google Play, foi possível ver que temos um problema em um aplicativo.

In [6]:
print(data_google[10472]) # exibe o registro 10472 do dataset

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


O aplicativo acima possui um erro em uma das colunas, por isso vamos remover essa linha do dataset.

In [7]:
del data_google[10472] # deleta o registro 10472 do dataset

### Removendo aplicativos duplicados

#### Checando se existem aplicativos duplicados no dataset do Google Play

In [8]:
duplicate_apps = list()
unique_apps = list()

for app in data_google: # para cada registro do dataset
    name = app[0] # pega o nome da app
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print('Total de aplicativos duplicados:', len(duplicate_apps)) # imprime a quantidade de aplicativos duplicados
print('Exemplo de aplicativos duplicados:', duplicate_apps[:10]) # imprime alguns aplicativos duplicados

Total de aplicativos duplicados: 1181
Exemplo de aplicativos duplicados: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


#### Checando se existem aplicativos duplicados no dataset da App Store

In [9]:
duplicate_apps = list()
unique_apps = list()

for app in data_apple: # para cada registro do dataset
    name = app[0] # pega o nome da app
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print('Total de aplicativos duplicados:', len(duplicate_apps)) # imprime a quantidade de aplicativos duplicados
print('Exemplo de aplicativos duplicados:', duplicate_apps[:10]) # imprime alguns aplicativos duplicados

Total de aplicativos duplicados: 0
Exemplo de aplicativos duplicados: []


Para remover os aplicativos duplicados encontrados no dataset do Google Play, vamos verificar a quantidade de avaliações que cada aplicativo duplicado possui. O aplicativo com maior quantidade de avaliações será mantido.

In [10]:
reviews_max = dict()

for app in data_google: # para cada registro do dataset
    name = app[0] # pega o nome do app
    reviews = float(app[3]) # pega a quantidade de reviews do app
    if name in reviews_max and reviews_max[name] < reviews:
        reviews_max[name] = reviews
    elif name not in reviews_max:
        reviews_max[name] = reviews

In [11]:
# Verificando se o tamanho do dicionário com os reviews mais altos é igual ao tamanho do dataset menos a quantidade de apps duplicados
print(len(reviews_max))
print(len(data_google) - 1181)

9659
9659


#### Utilizando o dicionário acima para remover os aplicativos duplicados

In [12]:
android_clean = list()
already_added = list()

for app in data_google: # para cada registro do dataset
    name = app[0] # pega o nome do app
    reviews = float(app[3]) # pega a quantidade de reviews do app
    if reviews == reviews_max[name] and name not in already_added: 
        android_clean.append(app) # adiciona o app ao dataset android_clean
        already_added.append(name) # adiciona o nome do app ao dataset already_added

In [13]:
print('Quantidade de aplicativos no dataset depois de limpo:', len(android_clean))
print(android_clean[:10])

Quantidade de aplicativos no dataset depois de limpo: 9659
[['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'], ['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'], ['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up'], ['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up'], ['Smoke Effect Photo Maker - Smoke E

### Removendo aplicativos que não são em inglês

Os números que correspondem aos caracteres que normalmente usamos em um texto em inglês estão todos no intervalo de 0 a 127, de acordo com o sistema ASCII (American Standard Code for Information Interchange).

Com base nesse intervalo, vamos construir uma função que detecta se um caractere pertence ou não ao conjunto de caracteres comuns do inglês. Se o número for igual ou menor que 127, o caractere é inglês.

Se o nome de um aplicativo tiver um caractere maior que 127, provavelmente significa que o aplicativo tem um nome diferente do inglês.

In [14]:
# Função que verifica se o app está escrito em inglês
def check_english(text): # text é o nome do app a ser verificado
    count = 0 # inicializa o contador com 0
    
    for word in text.strip(): 
        if ord(word) > 127: # se o caractere for maior que 127
            count += 1 # incrementa o contador
            
    if count > 3:
        return False
    else:
        return True

In [15]:
# Testando a função check_english
print('Está escrito em inglês?', check_english('Instagram'))
print('Está escrito em inglês?', check_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print('Está escrito em inglês?', check_english('Docs To Go™ Free Office Suite'))
print('Está escrito em inglês?', check_english('Instachat 😜'))

Está escrito em inglês? True
Está escrito em inglês? False
Está escrito em inglês? True
Está escrito em inglês? True


A função __check_english()__ é usada para filtrar os aplicativos que não estão em inglês para ambos os datasets:

In [16]:
data_google_english = list()
data_apple_english = list()

for app in android_clean: # para cada registro do dataset
    name = app[0] # pega o nome do app
    if check_english(name): # se o nome do app estiver escrito em inglês
        data_google_english.append(app) # adiciona o app ao dataset data_google_english
        
for app in data_apple: # para cada registro do dataset
    name = app[1] # pega o nome do app
    if check_english(name): # se o nome do app estiver escrito em inglês
        data_apple_english.append(app) # adiciona o app ao dataset data_apple_english

In [17]:
explore_data(data_google_english, 0, 3, True)
print('\n')
explore_data(data_apple_english, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Quantidade de linhas: 9614
Quantidade de colunas: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.1

### Isolando somentes os apps gratuitos
Os dois datasets possuem aplicativos gratuitos e pagos. Portanto, precisamos isolar apenas os aplicativos gratuitos para essa análise.

In [18]:
data_google_english_free = list()
data_apple_english_free = list()

for app in data_google_english: # para cada registro do dataset
    price = app[7] # pega o preço do app
    if price == '0': # se o preço for 0
        data_google_english_free.append(app) # adiciona o app ao dataset data_google_english_free

for app in data_apple_english: # para cada registro do dataset
    price = app[4] # pega o preço do app
    if price == '0.0': # se o preço for 0
        data_apple_english_free.append(app) # adiciona o app ao dataset data_apple_english_free

In [19]:
explore_data(data_google_english_free, 0, 3, True)
print('\n')
explore_data(data_apple_english_free, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Quantidade de linhas: 8864
Quantidade de colunas: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.1

Existem 8.864 aplicativos Android e 3.222 aplicativos iOS restantes, o que deve ser suficiente para a análise.

---

## Verificando os gêneros de apps mais comuns

Vamos começar a análise entendendo os gêneros mais comuns para cada loja de aplicativos. Para isso, criaremos uma tabela de frequência para a coluna __prime_genre__ da App Store e as colunas __Genres__ e __Category__ da Google Play.

Iremos criar duas funções que podemos usar para analisar as tabelas de frequência:
* Uma função para gerar tabelas de frequência que mostram porcentagens
* Outra função para exibir as porcentagens em ordem decrescente

In [20]:
def freq_table(dataset, index): # dataset é o dataset a ser analisado, index é o índice do campo a ser analisado
    table = {} # inicializa a tabela como um dicionário
    total = 0
    
    for row in dataset: # para cada registro do dataset
        total += 1 # incrementa o total
        value = row[index] # pega o valor do campo do registro
        if value in table: # se o valor do campo já estiver na tabela
            table[value] += 1 # incrementa o valor do campo na tabela
        else:
            table[value] = 1 # se o valor do campo não estiver na tabela, adiciona o valor do campo na tabela com o valor 1
    
    table_percentages = {} # inicializa a tabela como um dicionário
    for key in table: # para cada chave da tabela
        percentage = (table[key] / total) * 100 # calcula o percentual do valor do campo na tabela
        table_percentages[key] = percentage # adiciona o percentual do valor do campo na tabela
    
    return table_percentages # retorna a tabela com os percentuais


def display_table(dataset, index):
    table = freq_table(dataset, index) # chama a função freq_table para criar a tabela
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple) # adiciona o percentual e o valor do campo na tabela
        
    table_sorted = sorted(table_display, reverse = True) # ordena a tabela em ordem decrescente
    for entry in table_sorted:
        print(entry[1], ':', entry[0]) # imprime o valor do campo e o percentual do valor do campo

Começamos examinando a tabela de frequência da coluna __prime_genre__ da App Store.

In [21]:
display_table(data_apple_english_free, -5) # prime_genre é o índice da tabela a ser analisado

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


Podemos ver que entre os aplicativos gratuitos em inglês, mais da metade (58,16%) são games. Os aplicativos de entretenimento estão perto de 8%, seguidos pelos aplicativos de foto e vídeo, que estão perto de 5%. Apenas 3,66% dos aplicativos são feitos para educação, seguidos por aplicativos de redes sociais que representam 3,29% dos aplicativos.

Abaixo vamos continuar examinando as colunas __Genres__ e __Category__ do dataset da Google Play.

In [22]:
display_table(data_google_english_free, 1) # Category é o índice da tabela a ser analisado

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

No Google Play temos um maior volume de apps projetados para __Família (19%)__. Mas se investigarmos mais a fundo, podemos perceber que jogos para crianças se encaixam nessa categoria e são a maioria. Podemos notar ainda que temos uma boa gama de aplicativos voltados para coisas que ajudam no dia a dia. Exemplo: ferramentas, business, lifestyle, produtividade, finanças e etc.

In [23]:
display_table(data_google_english_free, -4) # Genres é o índice da tabela a ser analisado

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

A diferença entre as colunas Gêneros e Categorias não é muito clara, mas uma coisa que podemos notar é que a coluna Gêneros é muito mais granular (tem mais categorias). No momento, estamos apenas procurando uma visão geral, então trabalharemos apenas com a coluna Categoria daqui para frente.

Até aqui, descobrimos que a App Store é dominada por aplicativos projetados para diversão, enquanto a Google Play mostra um cenário mais equilibrado de aplicativos práticos e divertidos.

Nosso próximo passo é ter uma ideia sobre o tipo de aplicativo que tem mais usuários.

---

# Tipo de apps com maior volume de usuários

Uma maneira de descobrir quais gêneros são os mais populares (têm mais usuários), é calculando o número médio de instalações para cada gênero. No dataset da Google Play, encontramos essas informações na coluna __Installs__. Mas para a App Store, essa info está ausente. Como solução alternativa usaremos o número total de avaliações de usuários, que podemos encontrar na coluna __rating_count_tot__.

### App Store
Abaixo, calculamos o número médio de avaliações por gênero de aplicativo na App Store:

In [24]:
genres_ios = freq_table(data_apple_english_free, -5) # chama a função freq_table para criar a tabela

for genre in genres_ios: # para cada gênero da tabela
    total = 0
    len_genre = 0
    for app in data_apple_english_free: # para cada registro do dataset
        genre_app = app[-5] # pega o gênero do app
        if genre_app == genre: # se o gênero do app for igual ao gênero da tabela         
            n_ratings = float(app[5]) # pega o número de avaliações do app
            total += n_ratings # incrementa o total de avaliações
            len_genre += 1 # incrementa o total de apps com o gênero
    avg_n_ratings = total / len_genre # calcula a média de avaliações
    print(f'{genre}: {avg_n_ratings:.2f}') # imprime o gênero e a média de avaliações

Social Networking: 71548.35
Photo & Video: 28441.54
Games: 22788.67
Music: 57326.53
Reference: 74942.11
Health & Fitness: 23298.02
Weather: 52279.89
Utilities: 18684.46
Travel: 28243.80
Shopping: 26919.69
News: 21248.02
Navigation: 86090.33
Lifestyle: 16485.76
Entertainment: 14029.83
Food & Drink: 33333.92
Sports: 23008.90
Book: 39758.50
Finance: 31467.94
Education: 7003.98
Productivity: 21028.41
Business: 7491.12
Catalogs: 4004.00
Medical: 612.00


Em média, os aplicativos de mapas têm o maior número de reviews. Porém, esse número é fortemente influenciado pelo Waze e Google Maps, que juntos têm cerca de meio milhão de avaliações:

In [25]:
for app in data_apple_english_free:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5]) # imprime o nome do app e o número de avaliações

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


O mesmo padrão se aplica a aplicativos de redes sociais, onde o número médio é fortemente influenciado por alguns gigantes como Facebook, Pinterest, Skype, etc. Isso também se aplica a aplicativos de música, onde temos Pandora, Spotify e Shazam.

Nosso objetivo é encontrar gêneros populares, mas aplicativos de mapas, redes sociais ou música podem parecer mais populares do que realmente são.

Poderíamos obter uma imagem melhor removendo esses aplicativos extremamente populares para cada gênero e depois retrabalhando as médias, mas deixaremos esse nível de detalhe para mais tarde.

Os aplicativos de referência têm 74.942 avaliações em média, mas na verdade são a Bíblia e o Dictionary.com que distorcem a classificação média:

In [26]:
for app in data_apple_english_free:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


Este nicho parece mostrar algum potencial. Uma coisa que poderíamos fazer é pegar outro livro popular e transformá-lo em um aplicativo onde poderíamos adicionar recursos diferentes além da versão bruta do livro. Isso pode incluir citações diárias do livro, uma versão em áudio do livro, questionários sobre o livro e etc. Além disso, também podemos incorporar um dicionário no aplicativo, para que os usuários não precisem sair do aplicativo para procurar palavras que não conhecem.

### Google Play
Para o Google Play temos dados sobre o número de instalações, então podemos ter uma visão mais clara sobre a popularidade de cada gênero. No entanto, os números de instalação não parecem precisos o suficiente - podemos ver que a maioria dos valores é aberta (100+, 1.000+, 5.000+, etc):

In [27]:
display_table(data_google_english_free, 5) # Coluna 5 é o índice da tabela a ser analisado

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


Um problema com esses dados é que eles não são precisos. Por exemplo, não sabemos se um aplicativo com mais de 100.000 instalações tem 100.000 instalações, 200.000 ou 350.000. Porém, não precisamos de dados muito precisos para nossos propósitos - queremos apenas ter uma ideia de quais gêneros atraem mais usuários e não precisamos de precisão perfeita.

Vamos deixar os números como estão, o que significa que vamos considerar que um aplicativo com mais de 100.000 instalações possui 100.000 instalações e um aplicativo com mais de 1.000.000 instalações possui 1.000.000 instalações e assim por diante.

Para fazer os cálculos, precisaremos converter cada número de instalação em float — isso significa que temos que remover as vírgulas e os sinais de +, caso contrário vai dar erro. Faremos isso diretamente no loop abaixo, onde também calculamos o número médio de instalações para cada gênero (categoria).

In [28]:
categories_android = freq_table(data_google_english_free, 1) # chama a função freq_table para criar a tabela

for category in categories_android: # para cada categoria da tabela
    total = 0
    len_category = 0
    for app in data_google_english_free:
        category_app = app[1] # pega a categoria do app
        if category_app == category: # se a categoria do app for igual a categoria da tabela           
            n_installs = app[5] # pega o número de instalações do app
            n_installs = n_installs.replace(',', '') # remove a vírgula dos números de instalações
            n_installs = n_installs.replace('+', '') # remove o sinal de mais dos números de instalações
            total += float(n_installs) # incrementa o total de instalações
            len_category += 1 # incrementa o total de apps com a categoria
    avg_n_installs = total / len_category # calcula a média de instalações
    print(f'{category}: {avg_n_installs:.2f}') # imprime a categoria e a média de instalações

ART_AND_DESIGN: 1986335.09
AUTO_AND_VEHICLES: 647317.82
BEAUTY: 513151.89
BOOKS_AND_REFERENCE: 8767811.89
BUSINESS: 1712290.15
COMICS: 817657.27
COMMUNICATION: 38456119.17
DATING: 854028.83
EDUCATION: 1833495.15
ENTERTAINMENT: 11640705.88
EVENTS: 253542.22
FINANCE: 1387692.48
FOOD_AND_DRINK: 1924897.74
HEALTH_AND_FITNESS: 4188821.99
HOUSE_AND_HOME: 1331540.56
LIBRARIES_AND_DEMO: 638503.73
LIFESTYLE: 1437816.27
GAME: 15588015.60
FAMILY: 3695641.82
MEDICAL: 120550.62
SOCIAL: 23253652.13
SHOPPING: 7036877.31
PHOTOGRAPHY: 17840110.40
SPORTS: 3638640.14
TRAVEL_AND_LOCAL: 13984077.71
TOOLS: 10801391.30
PERSONALIZATION: 5201482.61
PRODUCTIVITY: 16787331.34
PARENTING: 542603.62
WEATHER: 5074486.20
VIDEO_PLAYERS: 24727872.45
NEWS_AND_MAGAZINES: 9549178.47
MAPS_AND_NAVIGATION: 4056941.77


Os aplicativos de comunicação possuem mais instalações: 38.456.119. Esse número é com certeza distorcido por alguns apps com mais de um bilhão de instalações (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail e Hangouts) e alguns outros com mais de 100 e 500 milhões de instalações:

In [29]:
for app in data_google_english_free:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                    or app[5] == '500,000,000+'
                                    or app[5] == '100,000,000+'):
        print(app[0], ':', app[5]) # imprime o nome do app e o número de instalações

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

Se deletarmos todos os aplicativos de comunicação com mais de 100 milhões de instalações, a média seria reduzida cerca de dez vezes:

In [30]:
under_100_m = [] # inicializa a lista com os apps com instalações menores que 100 milhões

for app in data_google_english_free:
    n_installs = app[5] # pega o número de instalações do app
    n_installs = n_installs.replace(',', '') # remove a vírgula dos números de instalações
    n_installs = n_installs.replace('+', '') # remove o sinal de mais dos números de instalações
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs)) # adiciona o número de instalações do app na lista
        
sum(under_100_m) / len(under_100_m) # calcula a média de instalações dos apps com instalações menores que 100 milhões

3603485.3884615386

Vemos o mesmo padrão para a categoria de players de vídeo, que é a segunda colocada com 24.727.872 instalações. O mercado é dominado por aplicativos como YouTube, Google Play Movies & TV ou MX Player. O padrão se repete para aplicativos sociais (onde temos gigantes como Facebook, Instagram, Google+, etc.), aplicativos de fotografia (Google Fotos entre outros) ou aplicativos de produtividade (Microsoft Word, Dropbox, Google Calendar, Evernote, etc).

Novamente, a principal preocupação é que esses gêneros de aplicativos possam parecer mais populares do que realmente são. Além disso, esses nichos parecem ser dominados por alguns gigantes com os quais é difícil competir.

Os livros e o gênero de referência também parecem bastante populares, com um número médio de instalações de 8.767.811. É interessante explorar isso com mais profundidade, pois descobrimos que esse gênero tem potencial para funcionar bem na App Store, e nosso objetivo é recomendar um gênero que mostre potencial para ser lucrativo tanto na App Store quanto na Google Play.

Vamos dar uma olhada em alguns dos aplicativos desse gênero e seu número de instalações:

In [31]:
for app in data_google_english_free:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5]) # imprime o nome do app e o número de instalações

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

O gênero livro e referência inclui uma variedade de aplicativos: software para processamento e leitura de e-books, várias coleções de bibliotecas, dicionários, tutoriais sobre programação e etc. Parece que ainda há um pequeno número de aplicativos extremamente populares que distorcem a média:

In [32]:
for app in data_google_english_free:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5]) # imprime o nome do app e o número de instalações

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


No entanto, parece que existem apenas alguns aplicativos muito populares, então esse mercado ainda mostra potencial. Vamos tentar obter algumas ideias de aplicativos com base nos tipos que estão em algum lugar no meio em termos de popularidade (entre 1.000.000 e 100.000.000 de downloads):

In [33]:
for app in data_google_english_free:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5]) # imprime o nome do app e o número de instalações

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

Esse nicho parece ser dominado por apps de leitura de e-books, além de várias coleções de bibliotecas e dicionários. Portanto, provavelmente não é uma boa ideia criar aplicativos semelhantes, pois haverá uma concorrência significativa.

Também notamos que existem alguns aplicativos criados em torno do livro Alcorão, o que sugere que criar um aplicativo em torno de um livro popular pode ser lucrativo. Parece que pegar um livro popular (talvez um livro mais recente) e transformá-lo em um aplicativo pode ser lucrativo para a Google Play e App Store.

Mas parece que o mercado já está cheio de bibliotecas, então precisamos adicionar alguns recursos especiais além da versão bruta do livro. Isso pode incluir citações diárias do livro, uma versão em áudio, testes sobre o livro, um fórum onde as pessoas podem discutir sobre o livro, etc.

---

## Conclusão

Nesse projeto, foram analisados dados sobre os apps da App Store e Google Play com o objetivo de recomendar um perfil de aplicativo que possa ser lucrativo para as duas lojas.

Ao longo das análise vimos que pegar um livro popular (talvez um livro mais recente) e transformá-lo em um aplicativo pode ser lucrativo para os mercados da Google Play e App Store. As duas lojas já estão cheias de bibliotecas, então precisamos adicionar alguns recursos especiais além da versão bruta do livro. Isso pode incluir citações diárias, uma versão em áudio, testes/exercícios, um fórum onde as pessoas podem discutir o livro, etc.

![](bot-reps-dataquest.jpg)